# CSCI 3155 Recitation 11

April 5, 2019

## Exercise: Types in concrete and abstract syntax
For the following program, add type annotations, write the resulting type of the entire expression, then convert it into abstract / concrete syntax.

* Annotated: (multiple correct solutions)
```
let y = function(x) x in
    let x = y(3) == 2 in function(f) f(5)
```

### Solution
* Annotated: (multiple correct solutions)
```
let y: num => num = function(x: num) x in
    let x: bool = y(3) == 2 in function(f: num => num => num) f(5)
```
* Result type: `(num => num => num) => num => num`
* Abstract Syntax:
```
    Let(
        "y",
        FunType(NumType, NumType),"
        FunDef(
            "x",
            NumType,
            Ident("x")
        ),
        Let(
            "x",
            BoolType,
            Eq(FunCall(Ident("Y"), Const(3)), Const(2)),
            FunDef(
                "f",
                FunType(NumType, FunType(NumType, NumType)),
                FunCall(Ident("f"), Const(5))
            )
        )
    )
```

## Exercise: Typechecking rules
Imagine we add references to our typed Lettuce:
$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType \\
& | & BoolType \\
& | & FunType(\mathbf{Type}, \mathbf{Type}) \\
& | & \color{red}{RefType(\mathbf{Type})} \\
\end{array} $$

Assume the type's concrete syntax is `ref(t)`

Write rules for checking the type of a `DeRef` and `AssignRef`.

Then, annotate the following program:
```
let x = NewRef(3) in function(y) AssignRef(x, y)
```

### Solution:
$$\newcommand\typeOf{\mathbf{typeOf}}$$
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$
$$\semRule{\typeOf(\texttt{expr}, \alpha) = \texttt{RefType(}\texttt{t)}}{ \typeOf(\texttt{DeRef(expr)}, \alpha) =  t}{deref-ok}$$


$$\semRule{\typeOf(\texttt{e1}, \alpha) = \texttt{RefType(}\texttt{t)}\ \typeOf(\texttt{e2}, \alpha) = t}{ \typeOf(\texttt{AssignRef(e1, e2)}, \alpha) =  t}{assignref-ok}$$

* Annotated:
```
let x: ref(num) = NewRef(3) in function(y: num) AssignRef(x, y)
```
* Result type: `num => num`
* Abstract Syntax:
```
Let(
    "x",
    RefType(NumType),
    NewRef(Const(3)),
    FunDef(
        "y",
        NumType,
        AssignRef(Ident(x), Ident(y))
    )
)
```

# REVIEW

## Exercise: Generics
**\[generics\]** **\[CPS\]** **\[fold\]**

Implement a `zip` function that turns two lists into a list of tuples. It should work for all types.

Next add a continuation to make it tail recursive.

Next, do it with a fold.

In [26]:
import scala.annotation.tailrec


val l1 = List(1, 2, 3, 4)
val l2 = List("a", "b", "c", "d")

// BEGIN SOLUTION
def zip[T1, T2](l1: List[T1], l2: List[T2]): List[(T1, T2)] = (l1, l2) match {
    case (h1 :: t1, h2 :: t2) => (h1, h2) :: zip(t1, t2)
    case _ => List()
}


@tailrec
def zip_continuation[T1, T2, TFinal](l1: List[T1], l2: List[T2])(k: (List[(T1, T2)]) => TFinal): TFinal = (l1, l2) match {
    case (h1 :: t1, h2 :: t2) => zip_continuation(t1, t2)(l => k((h1, h2) :: l))
    case _ => k(List())
}

def zip_fold[T1, T2](l1: List[T1], l2: List[T2]): List[(T1, T2)] = 
    l1.foldLeft((List[(T1, T2)](), l2))(
        (acc, el) => acc match {
            case (l3, h :: t) => ((el, h) :: l3, t)
        }
    )._1.reverse
// END SOLUTION

val l3: List[(Int, String)] = zip(l1, l2)
assert(zip(l1, l2) == l1.zip(l2))
assert(zip(l2, l1) == l2.zip(l1))

import scala.annotation.tailrec



l1: List[Int] = List(1, 2, 3, 4)
l2: List[String] = List("a", "b", "c", "d")
defined function zip
defined function zip_continuation
defined function zip_fold
l3: List[(Int, String)] = List((1, "a"), (2, "b"), (3, "c"), (4, "d"))

## Exercise: Generic folds
**\[generics\]**, **\[continuations\]**, **\[fold\]**

Implement `foldRight` and `foldLeft` supporting arbitrary types using tail-recursion (use continuations if needed). Feel free to add the extra parameter to the assert if you use continuations.

In [6]:
import scala.annotation.tailrec

@tailrec
// BEGIN SOLUTION
def foldLeft[TElem, TAcc](l: List[TElem], init_acc: TAcc)(f: (TAcc, TElem) => TAcc): TAcc =
    l match {
        case Nil => init_acc
        case el :: t => foldLeft(t, f(init_acc, el))( f)
    }
// END SOLUTION

assert(foldLeft(List(1, 2, 3), 0  )((acc, el) => acc + el)          == 6)
assert(foldLeft(List(1, 2, 3), "0")((acc, el) => acc + el.toString) == "0123")

@tailrec
// BEGIN SOLUTION
// The default `id` for parameter k is optional. If we exclude it we'd just need to
// pass it in during the test
def foldRight[TElem, TAcc](l: List[TElem], init_acc: TAcc)(f: (TElem, TAcc) => TAcc, k: TAcc => TAcc = id[TAcc](_)): TAcc =
    l match {
        case Nil => k(init_acc)
        case el :: t => foldRight(t, init_acc)(f, acc => k(f(el, acc)))
    }
def id[T](x: T): T = x
// END SOLUTION

assert(foldRight(List(1, 2, 3), 0  )((acc, el) => acc + el)          == 6)
assert(foldRight(List(1, 2, 3), "0")((el, acc) => acc + el.toString) == "0321")

import scala.annotation.tailrec


defined function foldLeft
defined function foldRight
defined function id

## Exercise: Loop to fold / map / filter
**\[fold map filter\]**

Convert the following loops to their functional equivalents:

In [2]:
val l = List(4, 6, 2, 8, 5, 12, 16)

var x = ""
var y = 4
var z = List(1, 2)
for (element <- l) {
    if (element % 4 == 0) {
        x += element.toString
        y = element
        z = element :: z
    }
}

// BEGIN SOLUTION
val (x_ans, y_ans, z_ans) = l
    .filter(element => element % 4 == 0)
    .foldLeft(("", 4, List(1, 2)))(
        (acc, el) => acc match {
            case (x, y, z) => (x + el.toString, el, el :: z)
        }
    )
// END SOLUTION

assert(x == x_ans && y == y_ans && z == z_ans)

l: List[Int] = List(4, 6, 2, 8, 5, 12, 16)
x: String = "481216"
y: Int = 16
z: List[Int] = List(16, 12, 8, 4, 1, 2)
x_ans: String = "481216"
y_ans: Int = 16
z_ans: List[Int] = List(16, 12, 8, 4, 1, 2)

In [11]:
val l = List(4, 6, 2, 8, 5, 12, 6)

var x = List[Int]()
for (element <- l) {
    if(element < 7) {
        x = (element * 3) :: x
    }
}
x = x.reverse

// BEGIN SOLUTION
val answer = l
    .filter(el => el < 7)
    .map(el => el * 3)
// END SOLUTION

assert(answer == x)

l: List[Int] = List(4, 6, 2, 8, 5, 12, 6)
x: List[Int] = List(12, 18, 6, 15, 18)
answer: List[Int] = List(12, 18, 6, 15, 18)